aqi prediction 


In [2]:
import pandas as pd


In [3]:
df=pd.read_csv('delhi_aqi.csv')
df.head()
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201664 entries, 0 to 201663
Data columns (total 25 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   datetime      201664 non-null  object 
 1   date          201664 non-null  object 
 2   year          201664 non-null  int64  
 3   month         201664 non-null  int64  
 4   day           201664 non-null  int64  
 5   hour          201664 non-null  int64  
 6   day_of_week   201664 non-null  object 
 7   is_weekend    201664 non-null  int64  
 8   season        201664 non-null  object 
 9   city          201664 non-null  object 
 10  station       201664 non-null  object 
 11  latitude      201664 non-null  float64
 12  longitude     201664 non-null  float64
 13  pm25          201664 non-null  float64
 14  pm10          201664 non-null  float64
 15  no2           201664 non-null  float64
 16  so2           201664 non-null  float64
 17  co            201664 non-null  float64
 18  o3  

In [4]:
df.isnull().sum()

datetime        0
date            0
year            0
month           0
day             0
hour            0
day_of_week     0
is_weekend      0
season          0
city            0
station         0
latitude        0
longitude       0
pm25            0
pm10            0
no2             0
so2             0
co              0
o3              0
temperature     0
humidity        0
wind_speed      0
visibility      0
aqi             0
aqi_category    0
dtype: int64

In [5]:
df['date']=pd.to_datetime(df['date'])
df['hour']=df['date'].dt.hour
df['month']=df['date'].dt.month

In [7]:
features=['pm25','pm10','wind_speed','temperature','humidity','hour','month']
x=df[features]
y=df['aqi']

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
x_train, x_test,y_train,y_test=train_test_split(x,y,test_size=0.2, random_state=42)
model=RandomForestRegressor(n_estimators=200)
model.fit(x_train, y_train)

,n_estimators,200
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [9]:
from sklearn.metrics import mean_absolute_error
preds=model.predict(x_test)
print('MAE:', mean_absolute_error(y_test,preds))

MAE: 0.002241340837527526


In [10]:
preds=model.predict(x_test)


In [15]:
new_data=pd.DataFrame([{'pm25':180,'pm10':250,'wind_speed':2,'temperature':30,'humidity':45,'hour':14,'month':11}])
predicted_aqi=model.predict(new_data)
print('predicted AQI:',predicted_aqi[0])

predicted AQI: 346.0


In [22]:
stations=df['station'].unique()
print(stations)

['Anand Vihar, Delhi' 'Jahangirpuri, Delhi' 'Wazirpur, Delhi'
 'Bawana, Delhi' 'ITO, Delhi' 'Punjabi Bagh, Delhi' 'Okhla Phase 2, Delhi'
 'Shadipur, Delhi' 'Rohini, Delhi' 'RK Puram, Delhi' 'Siri Fort, Delhi'
 'Dwarka Sec 8, Delhi' 'NSIT Dwarka, Delhi' 'Mandir Marg, Delhi'
 'Noida Sec 62' 'Noida Sec 125' 'Greater Noida' 'Gurugram Vikas Sadan'
 'Gurugram Sec 51' 'Faridabad Sec 16A' 'Faridabad New Town'
 'Ghaziabad Vasundhara' 'Ghaziabad Loni']


In [34]:
df['predicted_aqi']=model.predict(df[features])
print(df[['station','predicted_aqi']].head())
area_aqi=df.groupby('station',as_index=False)['predicted_aqi'].mean()
print(area_aqi)

               station  predicted_aqi
0   Anand Vihar, Delhi          500.0
1   Anand Vihar, Delhi          500.0
2   Anand Vihar, Delhi          500.0
3   Anand Vihar, Delhi          500.0
4  Jahangirpuri, Delhi          500.0
                 station  predicted_aqi
0     Anand Vihar, Delhi     295.166740
1          Bawana, Delhi     283.694234
2    Dwarka Sec 8, Delhi     248.200335
3     Faridabad New Town     260.427412
4      Faridabad Sec 16A     266.105066
5         Ghaziabad Loni     279.569124
6   Ghaziabad Vasundhara     270.586380
7          Greater Noida     248.490976
8        Gurugram Sec 51     256.009093
9   Gurugram Vikas Sadan     260.431473
10            ITO, Delhi     276.973962
11   Jahangirpuri, Delhi     289.718441
12    Mandir Marg, Delhi     247.955731
13    NSIT Dwarka, Delhi     244.716815
14         Noida Sec 125     255.898160
15          Noida Sec 62     263.219433
16  Okhla Phase 2, Delhi     274.130778
17   Punjabi Bagh, Delhi     270.277249
18       RK 

In [37]:
def classify_aqi(aqi):
    if aqi<=50:
        return 'good'
    elif aqi<=100:
        return 'satisfactory'
    elif aqi<=200:
        return 'moderate'
    elif aqi<=300:
        return 'poor'
    elif aqi<=400:
        return 'very poor'
    else:
        return 'severe'
    
area_aqi['category']=area_aqi['predicted_aqi'].apply(classify_aqi)
print(area_aqi)


                 station  predicted_aqi category
0     Anand Vihar, Delhi     295.166740     poor
1          Bawana, Delhi     283.694234     poor
2    Dwarka Sec 8, Delhi     248.200335     poor
3     Faridabad New Town     260.427412     poor
4      Faridabad Sec 16A     266.105066     poor
5         Ghaziabad Loni     279.569124     poor
6   Ghaziabad Vasundhara     270.586380     poor
7          Greater Noida     248.490976     poor
8        Gurugram Sec 51     256.009093     poor
9   Gurugram Vikas Sadan     260.431473     poor
10            ITO, Delhi     276.973962     poor
11   Jahangirpuri, Delhi     289.718441     poor
12    Mandir Marg, Delhi     247.955731     poor
13    NSIT Dwarka, Delhi     244.716815     poor
14         Noida Sec 125     255.898160     poor
15          Noida Sec 62     263.219433     poor
16  Okhla Phase 2, Delhi     274.130778     poor
17   Punjabi Bagh, Delhi     270.277249     poor
18       RK Puram, Delhi     255.608942     poor
19         Rohini, D

In [38]:
def infer_cause(row):
    if row['pm10']>150 and row['pm10']>row['pm25']*1.5:
        return 'road dust'
    elif row['no2']>60 :
        return 'traffic emission'
    elif row['so2']>40 :
        return 'industrial emission'
    elif row['pm25']>120:
        return 'combustion/domestic sources'
    else:
        return 'various factors'
df['inferred_cause']=df.apply(infer_cause, axis=1)

In [39]:
area_analysis=df.groupby('station').agg({'predicted_aqi':'mean','pm10':'mean','pm25':'mean','inferred_cause':lambda x:x.value_counts().idxmax()}).reset_index()

In [40]:
def intervention(cause):
    if cause=='road dust':
        return 'install water sprinklers'
    elif cause=='traffic emission':
        return 'promote EV and public transport'
    elif cause=='industrial emission':
        return 'enforce stricter emission standards and promote cleaner technologies'
    elif cause=='combustion/domestic sources':
        return 'promote cleaner cooking fuels and improve ventilation'
    else:
        return 'green buffer planatation'
area_analysis['solution']=area_analysis['inferred_cause'].apply(intervention)

we have different scales for aqi and pm10 and pm2.5 and no2 and so2 thus calculating severity index for uniformity( it is pollution intensity multiplier)

In [42]:
area_analysis['dust_index']=area_analysis['pm10']/area_analysis['pm10'].max()
area_analysis['pollution_index']=area_analysis['predicted_aqi']/area_analysis['predicted_aqi'].max()

1 mature tree offsets -20 kgs of CO2 per year 
acc to WHO recommendation 9 sq km green space per capita 
assume each area has 10,000 population

In [ ]:
#assuming base trees and max trees
base_trees=300
max_trees=1200
def calculate_trees(pollution_index):
    trees_needed=int(base_trees+(pollution_index*(max_trees-base_trees)))
    return round(trees_needed)
area_analysis['trees_required']=area_analysis['pollution_index'].apply(calculate_trees)

sprinklers truck calculation :  1 sprinkler truck covers 8 km distper day and (for our convenience in the project we have assumed the urban station covers 25 km road network)
thus Required trucks= (roadlength*frequency factor)/8

In [43]:
#assumed road length per area and truck coverage
road_length_per_area=25
truck_coverage=8
def calculate_truck_coverage(dust_index):
    frequency=1+dust_index # 1 to 2 times per day
    trucks_needed=int((road_length_per_area*frequency)/truck_coverage)
    return round(trucks_needed) 
area_analysis['sprinkler_trucks_needed']=area_analysis['dust_index'].apply(calculate_truck_coverage)

detailed budget for truck installation
1 truck=45 lakhs
annual operating cost=10 lakh
1 tree plantation (3 year maintenance)=2500

In [44]:
truck_cost=4500000
truck_operation=1000000
tree_cost=2500

def calculate_budget(row):
    if row['sprinkler_trucks_needed']>=1:
     sprinkler_cost=row['sprinkler_trucks_needed']*(truck_cost+truck_operation)
     total_cost+=sprinkler_cost
    if row['trees required']>=100:
     tree_cost=row['trees_required']*tree_cost
     total_cost+=tree_cost

    return total_cost


